# Lab 3.3 - Agentic AI with LangGraph + watsonx.ai

This notebook demonstrates how to build a stateful agent workflow using LangGraph with watsonx.ai.

## What You'll Learn

- How to create stateful agent workflows with LangGraph
- Integrating watsonx.ai models with LangGraph
- Building multi-node agent graphs with conditional routing
- Implementing RAG (Retrieval-Augmented Generation) pipelines
- Managing agent state across multiple steps

## Architecture

```
User Question
     |
     v
  Router Node (decides which path to take)
     |
     +----------+----------+
     |          |          |
     v          v          v
  RAG Node  Calculator  Direct Answer
     |          |          |
     +----------+----------+
     |
     v
 Generation Node (watsonx.ai)
     |
     v
Final Answer
```

---

## 1. Setup and Installation

### Google Colab Compatibility

This notebook works both locally and in Google Colab.

In [ ]:
# Check if running in Google Colab
try:
    import google.colab
    IN_COLAB = True
    print("✓ Running in Google Colab")
except ImportError:
    IN_COLAB = False
    print("✓ Running in local environment")

In [ ]:
# Install required packages
!pip install -q langgraph langchain-core langchain-ibm "ibm-watsonx-ai>=1.1.22" requests typing-extensions

## 2. Configure watsonx.ai Credentials

Set up your IBM watsonx.ai credentials.

### Required Credentials

1. **API Key** - Get from [IBM Cloud IAM](https://cloud.ibm.com/iam/apikeys)
2. **Project ID** - From your watsonx.ai project settings
3. **URL** - Regional endpoint (default: us-south)

In [ ]:
import os
from getpass import getpass

# Configuration for watsonx.ai
WATSONX_URL = os.getenv("WATSONX_URL", "https://us-south.ml.cloud.ibm.com")

if not os.getenv("WATSONX_APIKEY"):
    WATSONX_APIKEY = getpass("Enter your watsonx.ai API Key: ")
else:
    WATSONX_APIKEY = os.getenv("WATSONX_APIKEY")

if not os.getenv("WATSONX_PROJECT_ID"):
    WATSONX_PROJECT_ID = getpass("Enter your watsonx.ai Project ID: ")
else:
    WATSONX_PROJECT_ID = os.getenv("WATSONX_PROJECT_ID")

# Model configuration
LLM_MODEL_ID = os.getenv("LLM_MODEL_ID", "ibm/granite-3-8b-instruct")

# Accelerator API URL
ACCELERATOR_API_URL = os.getenv("ACCELERATOR_API_URL", "http://localhost:8000/ask")

print("✓ Configuration loaded")
print(f"  Model: {LLM_MODEL_ID}")
print(f"  URL: {WATSONX_URL}")
print(f"  RAG API: {ACCELERATOR_API_URL}")

## 3. Initialize watsonx.ai LLM

We'll use the LangChain integration for watsonx.ai.

In [ ]:
from langchain_ibm import WatsonxLLM
from langchain_core.prompts import ChatPromptTemplate

# Configure model parameters for better responses
model_params = {
    "decoding_method": "greedy",
    "max_new_tokens": 1500,
    "min_new_tokens": 1,
    "temperature": 0.3,
    "top_k": 50,
    "top_p": 1
}

# Initialize watsonx.ai LLM
llm = WatsonxLLM(
    model_id=LLM_MODEL_ID,
    url=WATSONX_URL,
    apikey=WATSONX_APIKEY,
    project_id=WATSONX_PROJECT_ID,
    params=model_params
)

print("✓ watsonx.ai LLM initialized successfully")

# Test the LLM
test_response = llm.invoke("What is AI?")
print(f"\nTest response: {test_response[:100]}...")

## 4. Define Graph State

LangGraph uses a typed state that flows through the graph nodes.

In [ ]:
from typing_extensions import TypedDict
from typing import Dict, Any, Optional

class GraphState(TypedDict):
    """
    State for our LangGraph agent.
    
    Attributes:
        input_text: Original user question
        question_type: Type of question (rag, calculator, direct)
        rag_answer: Response from RAG service (if applicable)
        calculation_result: Result from calculator (if applicable)
        intermediate_context: Any intermediate processing context
        final_answer: Polished final answer from LLM
        metadata: Additional metadata about the processing
    """
    input_text: str
    question_type: Optional[str]
    rag_answer: Optional[Dict[str, Any]]
    calculation_result: Optional[str]
    intermediate_context: Optional[str]
    final_answer: Optional[str]
    metadata: Optional[Dict[str, Any]]


print("✓ Graph state defined")

## 5. Define Node Functions

Each node in the graph performs a specific function.

### Router Node

The router determines which path the question should take.

In [ ]:
import re

def router_node(state: GraphState) -> Dict[str, Any]:
    """
    Route the question to the appropriate handler.
    
    Logic:
    - If question contains math expressions or numbers with operators -> calculator
    - If question asks about knowledge/facts -> RAG
    - Otherwise -> direct answer
    """
    question = state["input_text"].lower()
    
    # Check for mathematical expressions
    math_patterns = [
        r'\d+\s*[+\-*/]\s*\d+',  # Basic arithmetic
        r'calculate|compute|what is \d+',  # Math keywords
        r'\d+\s*\*\*\s*\d+',  # Power
    ]
    
    for pattern in math_patterns:
        if re.search(pattern, question):
            return {"question_type": "calculator"}
    
    # Check for knowledge/RAG questions
    rag_keywords = [
        'what is', 'explain', 'describe', 'tell me about',
        'how does', 'why', 'rag', 'retrieval', 'watsonx',
        'granite', 'ai', 'machine learning', 'llm'
    ]
    
    if any(keyword in question for keyword in rag_keywords):
        return {"question_type": "rag"}
    
    # Default to direct answer
    return {"question_type": "direct"}


print("✓ Router node defined")

### RAG Node

Calls the RAG service to retrieve relevant information.

In [ ]:
import requests
import json

def rag_node(state: GraphState) -> Dict[str, Any]:
    """
    Query the RAG service with the user's question.
    """
    question = state["input_text"]
    
    try:
        payload = {"question": question}
        resp = requests.post(ACCELERATOR_API_URL, json=payload, timeout=60)
        resp.raise_for_status()
        data = resp.json()
        
        return {
            "rag_answer": data,
            "intermediate_context": f"Retrieved answer from RAG service",
            "metadata": {"source": "rag_service", "success": True}
        }
    except Exception as e:
        return {
            "rag_answer": {"error": str(e)},
            "intermediate_context": f"Error calling RAG service: {str(e)}",
            "metadata": {"source": "rag_service", "success": False, "error": str(e)}
        }


print("✓ RAG node defined")

### Calculator Node

Performs safe arithmetic calculations.

In [ ]:
import ast
import operator as op

# Safe calculator implementation
_allowed_operators = {
    ast.Add: op.add,
    ast.Sub: op.sub,
    ast.Mult: op.mul,
    ast.Div: op.truediv,
    ast.Pow: op.pow,
}

def _eval_ast(node):
    if isinstance(node, ast.Num):  # Python 3.7
        return node.n
    if isinstance(node, ast.Constant):  # Python 3.8+
        return node.value
    if isinstance(node, ast.BinOp) and type(node.op) in _allowed_operators:
        return _allowed_operators[type(node.op)](_eval_ast(node.left), _eval_ast(node.right))
    if isinstance(node, ast.UnaryOp) and isinstance(node.op, (ast.UAdd, ast.USub)):
        value = _eval_ast(node.operand)
        return +value if isinstance(node.op, ast.UAdd) else -value
    raise ValueError("Unsupported expression")

def calculator_node(state: GraphState) -> Dict[str, Any]:
    """
    Extract and evaluate mathematical expressions from the question.
    """
    question = state["input_text"]
    
    # Try to extract mathematical expression
    # Look for patterns like "calculate X" or "what is X"
    patterns = [
        r'calculate[:\s]+(.+)',
        r'compute[:\s]+(.+)',
        r'what is[:\s]+(.+)',
        r'solve[:\s]+(.+)',
    ]
    
    expression = question
    for pattern in patterns:
        match = re.search(pattern, question.lower())
        if match:
            expression = match.group(1).strip()
            break
    
    # Clean up the expression
    expression = expression.strip('?!.')
    
    try:
        parsed = ast.parse(expression, mode="eval")
        result = _eval_ast(parsed.body)
        
        return {
            "calculation_result": str(result),
            "intermediate_context": f"Calculated: {expression} = {result}",
            "metadata": {"source": "calculator", "success": True, "expression": expression}
        }
    except Exception as e:
        return {
            "calculation_result": f"Error: {str(e)}",
            "intermediate_context": f"Failed to calculate: {expression}",
            "metadata": {"source": "calculator", "success": False, "error": str(e)}
        }


print("✓ Calculator node defined")

### Direct Answer Node

For simple questions that don't need RAG or calculation.

In [ ]:
def direct_node(state: GraphState) -> Dict[str, Any]:
    """
    Handle direct questions without tools.
    """
    return {
        "intermediate_context": "Question will be answered directly by the LLM",
        "metadata": {"source": "direct", "success": True}
    }


print("✓ Direct answer node defined")

### Generation Node

The final node that uses watsonx.ai to generate the answer.

In [ ]:
def generation_node(state: GraphState) -> Dict[str, Any]:
    """
    Use watsonx.ai to generate the final answer based on the state.
    """
    question = state["input_text"]
    question_type = state.get("question_type", "unknown")
    
    # Build context based on question type
    if question_type == "rag" and state.get("rag_answer"):
        rag_data = state["rag_answer"]
        answer = rag_data.get("answer") or rag_data.get("result") or "No answer available"
        citations = rag_data.get("citations") or rag_data.get("chunks") or []
        
        prompt_template = ChatPromptTemplate.from_template(
            "You are a helpful AI assistant powered by watsonx.ai.\n\n"
            "Question: {question}\n\n"
            "Context from knowledge base:\n{context}\n\n"
            "Citations: {citations}\n\n"
            "Please provide a clear, concise answer based on the context above. "
            "If the context doesn't fully answer the question, say so."
        )
        
        formatted = prompt_template.format_messages(
            question=question,
            context=answer,
            citations=json.dumps(citations, indent=2) if citations else "None"
        )
        
    elif question_type == "calculator" and state.get("calculation_result"):
        result = state["calculation_result"]
        
        prompt_template = ChatPromptTemplate.from_template(
            "You are a helpful AI assistant powered by watsonx.ai.\n\n"
            "Question: {question}\n\n"
            "Calculation result: {result}\n\n"
            "Please explain the calculation and present the answer clearly."
        )
        
        formatted = prompt_template.format_messages(
            question=question,
            result=result
        )
        
    else:
        # Direct answer
        prompt_template = ChatPromptTemplate.from_template(
            "You are a helpful AI assistant powered by watsonx.ai.\n\n"
            "Question: {question}\n\n"
            "Please provide a helpful, accurate answer."
        )
        
        formatted = prompt_template.format_messages(question=question)
    
    # Generate response
    prompt_text = formatted[0].content
    final_answer = llm.invoke(prompt_text)
    
    return {"final_answer": final_answer}


print("✓ Generation node defined")

## 6. Build the LangGraph

Now we'll connect all the nodes into a graph.

In [ ]:
from langgraph.graph import START, END, StateGraph

# Create the graph
graph = StateGraph(GraphState)

# Add nodes
graph.add_node("router", router_node)
graph.add_node("rag", rag_node)
graph.add_node("calculator", calculator_node)
graph.add_node("direct", direct_node)
graph.add_node("generation", generation_node)

# Add edges from START to router
graph.add_edge(START, "router")

# Add conditional edges from router to appropriate handler
def route_question(state: GraphState) -> str:
    """Determine which node to route to based on question type."""
    q_type = state.get("question_type", "direct")
    return q_type

graph.add_conditional_edges(
    "router",
    route_question,
    {
        "rag": "rag",
        "calculator": "calculator",
        "direct": "direct"
    }
)

# All paths lead to generation
graph.add_edge("rag", "generation")
graph.add_edge("calculator", "generation")
graph.add_edge("direct", "generation")

# Generation leads to END
graph.add_edge("generation", END)

# Compile the graph
app = graph.compile()

print("✓ LangGraph compiled successfully")

## 7. Visualize the Graph (Optional)

LangGraph can generate a visual representation of the workflow.

In [ ]:
# Try to display the graph structure
try:
    from IPython.display import Image, display
    display(Image(app.get_graph().draw_mermaid_png()))
except Exception as e:
    print(f"Could not display graph visualization: {e}")
    print("\nGraph structure:")
    print("START -> router -> [rag|calculator|direct] -> generation -> END")

## 8. Test the Agent

Let's test our LangGraph agent with different types of questions.

### Test 1: Knowledge Question (RAG Path)

In [ ]:
question_1 = "What is Retrieval-Augmented Generation and why is it important?"

print("=" * 80)
print(f"QUESTION: {question_1}")
print("=" * 80)

state_1 = {"input_text": question_1}
result_1 = app.invoke(state_1)

print(f"\nRoute taken: {result_1.get('question_type', 'unknown')}")
print(f"Intermediate: {result_1.get('intermediate_context', 'N/A')}")
print("\n" + "=" * 80)
print("FINAL ANSWER:")
print("=" * 80)
print(result_1.get('final_answer', 'No answer generated'))

### Test 2: Math Question (Calculator Path)

In [ ]:
question_2 = "Calculate: (25 + 15) * 2 - 10"

print("=" * 80)
print(f"QUESTION: {question_2}")
print("=" * 80)

state_2 = {"input_text": question_2}
result_2 = app.invoke(state_2)

print(f"\nRoute taken: {result_2.get('question_type', 'unknown')}")
print(f"Intermediate: {result_2.get('intermediate_context', 'N/A')}")
print("\n" + "=" * 80)
print("FINAL ANSWER:")
print("=" * 80)
print(result_2.get('final_answer', 'No answer generated'))

### Test 3: General Question (Direct Path)

In [ ]:
question_3 = "Hello! How can you help me today?"

print("=" * 80)
print(f"QUESTION: {question_3}")
print("=" * 80)

state_3 = {"input_text": question_3}
result_3 = app.invoke(state_3)

print(f"\nRoute taken: {result_3.get('question_type', 'unknown')}")
print(f"Intermediate: {result_3.get('intermediate_context', 'N/A')}")
print("\n" + "=" * 80)
print("FINAL ANSWER:")
print("=" * 80)
print(result_3.get('final_answer', 'No answer generated'))

## 9. Interactive Agent Function

Create a helper function for easy interaction.

In [ ]:
def ask_agent(question: str, verbose: bool = True):
    """
    Ask a question to the LangGraph agent.
    
    Args:
        question: Your question
        verbose: Whether to print detailed information
    
    Returns:
        The final answer
    """
    if verbose:
        print("=" * 80)
        print(f"QUESTION: {question}")
        print("=" * 80)
    
    state = {"input_text": question}
    result = app.invoke(state)
    
    if verbose:
        print(f"\n📍 Route: {result.get('question_type', 'unknown').upper()}")
        print(f"🔄 Process: {result.get('intermediate_context', 'N/A')}")
        
        metadata = result.get('metadata', {})
        if metadata:
            print(f"📊 Metadata: {json.dumps(metadata, indent=2)}")
        
        print("\n" + "=" * 80)
        print("💡 ANSWER:")
        print("=" * 80)
        print(result.get('final_answer', 'No answer generated'))
        print("="* 80)
    
    return result.get('final_answer', 'No answer generated')


# Try it out!
# ask_agent("Your question here")

## 10. Advanced: Stream the Agent Execution

LangGraph supports streaming to see each step as it happens.

In [ ]:
def ask_agent_stream(question: str):
    """
    Ask a question and stream the execution step by step.
    """
    print("=" * 80)
    print(f"QUESTION: {question}")
    print("=" * 80)
    print("\n🔄 Execution Flow:\n")
    
    state = {"input_text": question}
    
    for step_num, step in enumerate(app.stream(state), 1):
        node_name = list(step.keys())[0]
        node_output = step[node_name]
        
        print(f"Step {step_num}: {node_name.upper()}")
        
        # Show relevant updates
        if "question_type" in node_output:
            print(f"  └─ Routed to: {node_output['question_type']}")
        if "intermediate_context" in node_output:
            print(f"  └─ {node_output['intermediate_context']}")
        if "final_answer" in node_output:
            print(f"  └─ Generated final answer")
        
        print()
    
    print("=" * 80)
    print("💡 FINAL ANSWER:")
    print("=" * 80)
    
    # Get final result
    final_result = app.invoke(state)
    print(final_result.get('final_answer', 'No answer generated'))
    print("=" * 80)


# Example
# ask_agent_stream("Explain what watsonx.ai is")

## 11. Summary and Key Takeaways

### What We Learned

1. **LangGraph Fundamentals**: Built a stateful agent workflow with multiple nodes
2. **Conditional Routing**: Implemented intelligent routing based on question type
3. **watsonx.ai Integration**: Used Granite models for response generation
4. **State Management**: Tracked state across multiple processing steps
5. **RAG Pipeline**: Integrated external knowledge retrieval

### Key Components

- **StateGraph**: The main graph structure that manages flow
- **Nodes**: Individual processing units (router, rag, calculator, generation)
- **Edges**: Connections between nodes (conditional and direct)
- **State**: Typed dictionary that flows through the graph

### Advantages of LangGraph

1. **Explicit Control Flow**: Clear, visual representation of agent logic
2. **State Management**: Built-in state tracking across steps
3. **Debugging**: Easy to inspect state at each node
4. **Streaming**: Can stream execution for real-time feedback
5. **Flexibility**: Easy to add new nodes or modify routing logic

### Best Practices

1. **Clear State Schema**: Define a comprehensive state type
2. **Single Responsibility**: Each node should do one thing well
3. **Error Handling**: Include try-catch blocks in nodes
4. **Metadata Tracking**: Add metadata for debugging and analysis
5. **Modular Design**: Keep nodes independent and reusable

### Comparison with Other Frameworks

| Feature | LangGraph | CrewAI | Plain LangChain |
|---------|-----------|--------|------------------|
| State Management | ✅ Built-in | ⚠️ Manual | ❌ Manual |
| Visual Flow | ✅ Yes | ❌ No | ❌ No |
| Conditional Routing | ✅ Native | ⚠️ Limited | ✅ Manual |
| Streaming | ✅ Yes | ✅ Yes | ✅ Yes |
| Learning Curve | Medium | Easy | Easy |

### When to Use LangGraph

- Complex multi-step workflows
- Need for explicit state management
- Conditional routing between different paths
- Debugging and visualization requirements
- Production-grade agent systems

### Next Steps

- Explore the CrewAI notebook for team-based agents
- Try Langflow for visual agent building
- Add more nodes (web search, database query, etc.)
- Implement memory/conversation history
- Build more complex conditional logic

---

**Course**: Multi-Agent Systems with watsonx.ai  
**Lab**: 3.3 - LangGraph Integration  
**Platform**: Compatible with Google Colab and local environments